# Chain-Of-Thought without Parser

We define several meta task to be solved and finally get the final decision of whether an text content is a "reminder".

In [ ]:
from langchain import LLMChain, PromptTemplate
from langchain.llms import OpenAI

# 1. Define a prompt to let the LLM know how to parse the emails
template_parse = """
You are an expert in email parsing. I will provide you with a text that contains one or more email messages.
Please determine how many emails are in the text. If there is only one email, return "1" and the email content.
If there are multiple emails, return "multiple" and the content of the first email.

Text: {email_text}
"""

prompt_parse = PromptTemplate(input_variables=["email_text"], template=template)
llm = OpenAI(temperature=0)


# 2. Define a prompt to verify if the extracted email is the ****first**** in the text
template_validation = """
You are an email validation expert. I will provide you with two parts: the entire text containing multiple emails and the extracted first email.
Please confirm if the extracted email is indeed the first email in the text.

Full text: {email_text}
Extracted email: {extracted_email}

If the extracted email matches the first email in the full text, return "Yes". Otherwise, return "No".
"""

prompt_validation = PromptTemplate(input_variables=["email_text", "extracted_email"], template=template_validation)

# 3. Define a prompt to check if the email is sent to a Foyer employee
template_check_foyer = """
You are an email analysis expert. I will provide you with an email, and your task is to determine if the recipient is an employee of the Foyer company.
You should check the email address to see if it ends with "foyer.lu", or check if the email content clearly indicates that the recipient is an internal Foyer employee or if the email was sent to Foyer itself.

Email content: {email_content}

If the recipient is a Foyer employee, return "Yes". If not, return "No", and explain why.
"""

prompt_check_foyer = PromptTemplate(input_variables=["email_content"], template=template_check_foyer)


# 4. Define a prompt to check if the email is a reminder to Foyer Insurance
template_cls_reminder = """
You are an expert in analyzing emails. I will provide you with an email sent to Foyer Insurance or one of its employees.
Your task is to determine if the email content qualifies as a "reminder." A reminder typically meets one of the following conditions:
1. It asks Foyer to reply to a previously sent email that has not yet received a response.
2. It reminds Foyer to proceed with the payment of an insurance claim.
3. It follows up on a previous request or email.
4. It inquires about the status of an incomplete insurance claim.

Based on these conditions, determine if the email is a reminder. If it is a reminder, return "Yes" and specify which condition it satisfies. If it is not a reminder, return "No".

Email content: {email_content}
"""
prompt_cls_reminder = PromptTemplate(input_variables=["email_content"], template=template_cls_reminder)


# 5. Define a prompt to validate the reminder response
template_validate_reminder = """
You are an expert in analyzing emails. Below is the original question about whether an email is a reminder, along with the answer provided.
Please evaluate if the answer is correct. If the answer is incorrect, correct it and provide a brief explanation.

Original question: {original_prompt}
Response: {response}

"""

prompt_validate_reminder = PromptTemplate(
    input_variables=["original_prompt", "response"], 
    template=template_validate_reminder
)
